In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql import *
from os import listdir
from os.path import isfile, join
from pyspark.sql.types import *
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [2]:
import nltk
nltk.download('inaugural')
nltk.download('stopwords')

[nltk_data] Downloading package inaugural to /root/nltk_data...
[nltk_data] Unzipping corpora/inaugural.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Unzipping corpora/stopwords.zip.
 Out[ 67 ]: True

In [3]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english'))

In [4]:
df = spark.sql("select * from smsspamcollection_txt")
# pdf = df.select("*").toPandas()

In [5]:
df.printSchema()

root
-- _c0: string (nullable = true)
-- _c1: string (nullable = true)

In [6]:
hamList = df[df["_c0"] == "ham"]
spamList = df[df["_c0"] == "spam"]

In [7]:
hamListFeatures  = hamList.select("_c1")
spamListFeatures = spamList.select("_c1")
hamListLabels  = hamList["_c0"]
spamListLabels = spamList["_c0"]
print(type(spamListFeatures))

<class 'pyspark.sql.dataframe.DataFrame'>

In [8]:
hamListRDD = hamListFeatures.rdd
spamListRDD = spamListFeatures.rdd
type(hamListRDD)
print(type(spamListRDD))
print(spamListRDD.collect())

<class 'pyspark.rdd.RDD'>
[Row(_c1="Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"), Row(_c1="FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv"), Row(_c1='WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'), Row(_c1='Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030'), Row(_c1='SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info'), Row(_c1='URGENT! You have won a 1 week FREE membership in our £100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18'), Row(_c1='XXXMobileMovieClub: To use your credit, click the WAP link in the next txt message or click here>> http://wap. xxxmobilemovieclub.com?n=QJKGIGHJJGCBL'), Row(_c1='England v Macedonia - dont miss the goals/team news. Txt ur national team to 87077 eg ENGLAND to 87077 Try:WALES, SCOTLAND 4txt/ú1.20 POBOXox36504W45WQ 16+'), Row(_c1='Thanks for your subscription to Ringtone UK your mobile will be charged £5/month Please confirm by replying YES or NO. If you reply NO you will not be charged'), Row(_c1='07732584351 - Rodger Burns - MSG = We tried to call you re your reply to our sms for a free nokia mobile + free camcorder. Please call now 08000930705 for delivery tomorrow'), Row(_c1='SMS. ac Sptv: The New Jersey Devils and the Detroit Red Wings play Ice Hockey. Correct or Incorrect? End? Reply END SPTV'), Row(_c1='Congrats! 1 year special cinema pass for 2 is yours. call 09061209465 now! C Suprman V, Matrix3, StarWars3, etc all 4 FREE! bx420-ip4-5we. 150pm. Dont miss out! '), Row(_c1='As a valued customer, I am pleased to advise you that following recent review of your Mob No. you are awarded with a £1500 Bonus Prize, call 09066364589'), Row(_c1='Urgent UR awarded a complimentary trip to EuroDisinc Trav, Aco&Entry41 Or £1000. To claim txt DIS to 87121 18+6*£1.50(moreFrmMob. ShrAcomOrSglSuplt)10, LS1 3AJ'), Row(_c1='Did you hear about the new "Divorce Barbie"? It comes with all of Ken\'s stuff!'), Row(_c1='Please call our customer service representative on 0800 169 6031 between 10am-9pm as you have WON a guaranteed £1000 cash or £5000 prize!'), Row(_c1='Your free ringtone is waiting to be collected. Simply text the password "MIX" to 85069 to verify. Get Usher and Britney. FML, PO Box 5249, MK17 92H. 450Ppw 16'), Row(_c1='GENT! We are trying to contact you. Last weekends draw shows that you won a £1000 prize GUARANTEED. Call 09064012160. Claim Code K52. Valid 12hrs only. 150ppm'), Row(_c1='You are a winner U have been specially selected 2 receive £1000 or a 4* holiday (flights inc) speak to a live operator 2 claim 0871277810910p/min (18+) '), Row(_c1='PRIVATE! Your 2004 Account Statement for 07742676969 shows 786 unredeemed Bonus Points. To claim call 08719180248 Identifier Code: 45239 Expires'), Row(_c1='URGENT! Your Mobile No. was awarded £2000 Bonus Caller Prize on 5/9/03 This is our final try to contact U! Call from Landline 09064019788 BOX42WR29C, 150PPM'), Row(_c1='Todays Voda numbers ending 7548 are selected to receive a $350 award. If you have a match please call 08712300220 quoting claim code 4041 standard rates app'), Row(_c1='Sunshine Quiz Wkly Q! Win a top Sony DVD player if u know which country the Algarve is in? Txt ansr to 82277. £1.50 SP:Tyrone'), Row(_c1="Want 2 get laid tonight? Want real Dogging locations sent direct 2 ur mob? Join the UK's largest Dogging Network bt Txting GRAVEL to 69888! Nt. ec2a. 31p.msg@150p"), Row(_c1="You'll not rcv any more msgs from the chat svc. For FREE Hardcore services text GO to: 69988 If

In [9]:
spamListFeat = []
hamListFeat = []
for row in spamListRDD.collect():
  spamListFeat.append(row["_c1"].split())
# print(spamListFeat)
filtered_sentence_spam = map(lambda words: [w for w in words if not w in stop_words] , spamListFeat)
FinalSpamrdd = sc.parallelize(filtered_sentence_spam) 
# print(FinalSpamrdd.collect())
for row in hamListRDD.collect():
  hamListFeat.append(row["_c1"].split())
filtered_sentence_ham = map(lambda words: [w for w in words if not w in stop_words] , hamListFeat)
FinalHamrdd = sc.parallelize(filtered_sentence_ham) 
print(FinalHamrdd.collect())


[['Go', 'jurong', 'point,', 'crazy..', 'Available', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet...', 'Cine', 'got', 'amore', 'wat...'], ['Ok', 'lar...', 'Joking', 'wif', 'u', 'oni...'], ['U', 'dun', 'say', 'early', 'hor...', 'U', 'c', 'already', 'say...'], ['Nah', 'I', 'think', 'goes', 'usf,', 'lives', 'around', 'though'], ['Even', 'brother', 'like', 'speak', 'me.', 'They', 'treat', 'like', 'aids', 'patent.'], ['As', 'per', 'request', "'Melle", 'Melle', '(Oru', 'Minnaminunginte', 'Nurungu', "Vettam)'", 'set', 'callertune', 'Callers.', 'Press', '*9', 'copy', 'friends', 'Callertune'], ["I'm", 'gonna', 'home', 'soon', 'want', 'talk', 'stuff', 'anymore', 'tonight,', 'k?', "I've", 'cried', 'enough', 'today.'], ["I've", 'searching', 'right', 'words', 'thank', 'breather.', 'I', 'promise', 'wont', 'take', 'help', 'granted', 'fulfil', 'promise.', 'You', 'wonderful', 'blessing', 'times.'], ['I', 'HAVE', 'A', 'DATE', 'ON', 'SUNDAY', 'WITH', 'WILL!!'], ['Oh', "k...i'm", 'watching', 'here:)'], ['Eh', 'u', 'remember', '2', 'spell', 'name...', 'Yes', 'did.', 'He', 'v', 'naughty', 'make', 'v', 'wet.'], ['Fine', 'that\x92s', 'way', 'u', 'feel.', 'That\x92s', 'way', 'gota', 'b'], ['Is', 'seriously', 'spell', 'name?'], ['I‘m', 'going', 'try', '2', 'months', 'ha', 'ha', 'joking'], ['So', 'ü', 'pay', 'first', 'lar...', 'Then', 'da', 'stock', 'comin...'], ['Aft', 'finish', 'lunch', 'go', 'str', 'lor.', 'Ard', '3', 'smth', 'lor.', 'U', 'finish', 'ur', 'lunch', 'already?'], ['Ffffffffff.', 'Alright', 'way', 'I', 'meet', 'sooner?'], ['Just', 'forced', 'eat', 'slice.', "I'm", 'really', 'hungry', 'tho.', 'This', 'sucks.', 'Mark', 'getting', 'worried.', 'He', 'knows', "I'm", 'sick', 'I', 'turn', 'pizza.', 'Lol'], ['Lol', 'always', 'convincing.'], ['Did', 'catch', 'bus', '?', 'Are', 'frying', 'egg', '?', 'Did', 'make', 'tea?', 'Are', 'eating', "mom's", 'left', 'dinner', '?', 'Do', 'feel', 'Love', '?'], ["I'm", 'back', '&amp;', "we're", 'packing', 'car', 'now,', "I'll", 'let', 'know', "there's", 'room'], ['Ahhh.', 'Work.', 'I', 'vaguely', 'remember', 'that!', 'What', 'feel', 'like?', 'Lol'], ['Wait', "that's", 'still', 'clear,', 'sure', 'sarcastic', "that's", 'x', 'want', 'live', 'us'], ['Yeah', 'got', '2', 'v', 'apologetic.', 'n', 'fallen', 'actin', 'like', 'spoilt', 'child', 'got', 'caught', 'that.', 'Till', '2!', 'But', 'go', 'there!', 'Not', 'badly', 'cheers.', 'You?'], ['K', 'tell', 'anything', 'you.'], ['For', 'fear', 'fainting', 'housework', 'did?', 'Quick', 'cuppa'], ['Yup...', 'Ok', 'go', 'home', 'look', 'timings', 'msg', 'ü', 'again...', 'Xuhui', 'going', 'learn', '2nd', 'may', 'lesson', '8am'], ['Oops,', "I'll", 'let', 'know', "roommate's", 'done'], ['I', 'see', 'letter', 'B', 'car'], ['Anything', 'lor...', 'U', 'decide...'], ['Hello!', "How's", 'saturday', 'go?', 'I', 'texting', 'see', 'decided', 'anything', 'tomo.', 'Not', "i'm", 'trying', 'invite', 'anything!'], ['Pls', 'go', 'ahead', 'watts.', 'I', 'wanted', 'sure.', 'Do', 'great', 'weekend.', 'Abiola'], ['Did', 'I', 'forget', 'tell', '?', 'I', 'want', ',', 'I', 'need', 'you,', 'I', 'crave', '...', 'But', '...', 'I', 'love', 'sweet', 'Arabian', 'steed', '...', 'Mmmmmm', '...', 'Yummy'], ['WHO', 'ARE', 'YOU', 'SEEING?'], ['Great!', 'I', 'hope', 'like', 'man', 'well', 'endowed.', 'I', '&lt;#&gt;', 'inches...'], ['No', 'calls..messages..missed', 'calls'], ["Didn't", 'get', 'hep', 'b', 'immunisation', 'nigeria.'], ['Fair', 'enough,', 'anything', 'going', 'on?'], ['Yeah', 'hopefully,', 'tyler', "can't", 'I', 'could', 'maybe', 'ask', 'around', 'bit'], ['U', 'know', 'stubborn', 'I', 'am.', 'I', 'even', 'want', 'go', 'hospital.', 'I', 'kept', 'telling', 'Mark', "I'm", 'weak', 'sucker.', 'Hospitals', 'weak', 'suckers.'], ['What', 'thinked', 'me.', 'First', 'time', 'saw', 'class.'], ['A', 'gram', 'usually', 'runs', 'like', '&lt;#&gt;', ',', 'half', 'eighth', 'smarter', 'though', 'gets', 'almost', 'whole', 'second', 'gram', '&lt;#&gt;'], ['K', 'fyi', 'x', 'ride', 'early', 'tomorrow', 'morning', 

In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
tansformHam = FinalHamrdd.toDF()
tansformSpam = FinalSpamrdd.toDF()
tansformHam.printSchema()
# assembler = VectorAssembler(
#     inputCols=["_1","_2","_3","_4","_5","_6","_7","_8","_9","_10","_11","_12","_13","_14","_15","_16","_17","_18","_19","_20"],
#     outputCol="features")
# output = assembler.transform(tansformHam)
# output.select("features").show(truncate=False)

root
-- _1: string (nullable = true)
-- _2: string (nullable = true)
-- _3: string (nullable = true)
-- _4: string (nullable = true)
-- _5: string (nullable = true)
-- _6: string (nullable = true)
-- _7: string (nullable = true)
-- _8: string (nullable = true)
-- _9: string (nullable = true)
-- _10: string (nullable = true)
-- _11: string (nullable = true)
-- _12: string (nullable = true)
-- _13: string (nullable = true)
-- _14: string (nullable = true)
-- _15: string (nullable = true)
-- _16: string (nullable = true)

In [11]:
from pyspark.ml.feature import CountVectorizer
# tf = HashingTF(numFeatures = 1000)
# spam_features = tf.transform(tansformHam)
# ham_features = tf.transform(tansformSpam)
vectorizer = CountVectorizer(inputCol = ["_1","_2","_3","_4","_5","_6","_7","_8","_9","_10","_11","_12","_13","_14","_15","_16","_17","_18","_19","_20"],
    outputCol="features")
vectorizer_transformerHam = vectorizer.fit(tansformHam)
# vectorizer_transformerSpam = vectorizer.fit(tansformSpam)

In [12]:
spam_samples = vectorizer_transformerSpam.map(lambda features:LabeledPoint(1, features))
ham_samples = vectorizer_transformerHam.map(lambda features:LabeledPoint(0, features))

In [13]:
samples = spam_samples.union(ham_samples)
[training_data, test_data] = samples.randomSplit([0.8, 0.2])

In [14]:
algo = LogisticRegressionWithSGD()
model = algo.train(training_data)
score(model)

In [15]:
algo = RandomForest()
model = algo.trainClassifier(training_data,numClasses=2,categoricalFeaturesInfo={},numTrees=16)
score(model)
